# Fine-Tuning a Pre-Trained Model 

GOALS: 
- How to prepare a large dataset from the Hub
- How to use Keras to fine-tune a model
- How to use Keras to get predictions
- How to use a custom metric

## Processing the Data 

Here is how to train a sequence classifier on one batch in tensorflow

In [1]:
import tensorflow as tf 
import numpy as np 
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [2]:
# Must be done at the very start, before any other TensorFlow operations
physical_devices = tf.config.list_physical_devices('GPU')
try:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("Memory growth enabled")
except RuntimeError as e:
    print(f"Error enabling memory growth: {e}")


Memory growth enabled


In [3]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

2024-12-19 13:08:38.205518: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-19 13:08:38.205542: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-19 13:08:38.205547: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-19 13:08:38.205581: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-19 13:08:38.205787: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized fr

In [4]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

In [5]:
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors='tf'))
batch

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
 array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
         12172,  2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [7]:
labels = tf.convert_to_tensor([1, 1])
labels

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 1], dtype=int32)>

In [8]:
model.train_on_batch(batch, labels)

2024-12-19 13:08:44.256483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


15.49500846862793

In this section we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset, introduced in a paper by William B. Dolan and Chris Brockett. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing). We’ve selected it for this chapter because it’s a small dataset, so it’s easy to experiment with training on it.

In [9]:
from datasets import load_dataset

In [10]:
# This command downloads and caches the dataset, by default in ~/.cache/huggingface/datasets
raw_datasets = load_dataset('glue','mrpc')
raw_datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [11]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[1]

{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
 'label': 0,
 'idx': 1}

We can see the labels are already integers, so we won’t have to do any preprocessing there. To know which integer corresponds to which label, we can inspect the features of our raw_train_dataset. This will tell us the type of each column:

In [12]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Behind the scenes, label is of type ClassLabel, and the mapping of integers to label name is stored in the names folder. 0 corresponds to `not_equivalent`, and 1 corresponds to `equivalent`.

In [13]:
raw_val_dataset = raw_datasets['validation']
print(raw_val_dataset[15], "\n""\n" , raw_val_dataset[87])

{'sentence1': 'Unable to find a home for him , a judge told mental health authorities they needed to find supervised housing and treatment for DeVries somewhere in California .', 'sentence2': 'The judge had told the state Department of Mental Health to find supervised housing and treatment for DeVries somewhere in California .', 'label': 1, 'idx': 127} 

 {'sentence1': 'However , EPA officials would not confirm the 20 percent figure .', 'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .', 'label': 0, 'idx': 812}


### Preprocessing a Dataset
To preprocess the dataset, we need to convert the text to numbers the model can make sense of.We can feed the tokenizer one sentence or a list of sentences, so we can directly tokenize all the first sentences and all the second sentences of each pair.

In [14]:
tokenized_sentences_1 = tokenizer(raw_datasets['train']['sentence1'])
tokenized_sentences_2 = tokenizer(raw_datasets['train']['sentence2'])

In [15]:
raw_datasets['train']['sentence1'][0:10]

['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .',
 'Revenue in the first quarter of the year dropped 15 percent from the same period a year earlier .',
 'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .',
 'The DVD-CCA then appealed to the state Supreme Court .',
 'That compared with $ 35.18 million , or 24 cents per share , in the year-ago period .',
 'Shares of Genentech , a much larger company with several products on the market , rose more than 2 percent .

However, we can’t just pass two sequences to the model and get a prediction of whether the two sentences are paraphrases or not. We need to handle the two sequences as a pair, and apply the appropriate preprocessing. Fortunately, the tokenizer can also take a pair of sequences and prepare it the way our BERT model expects:

In [16]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

`token_type_ids` is what tells the model which part of the input is the first sentence and which is the second sentence.m

Now we Take element 15 of the training set and tokenize the two sentences separately and as a pair. What’s the difference between the two results

In [17]:
tokenized_sentences_1 = tokenizer(raw_train_dataset[15]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_train_dataset[15]["sentence2"])

In [18]:
tokenized_sentences_1

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenized_sentences_2

{'input_ids': [101, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenized_sentences_pair = tokenizer(raw_train_dataset[15]["sentence1"], raw_train_dataset[15]["sentence2"])

In [21]:
tokenized_sentences_pair

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

If we decode the IDs inside input_ids back to words:

In [22]:
tokenizer.convert_ids_to_tokens(inputs['input_ids']) 

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

So we see the model expects the inputs to be of the form [CLS] sentence1 [SEP] sentence2 [SEP] when there are two sentences. Aligning this with the token_type_ids gives us:

In [23]:
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

As you can see, the parts of the input corresponding to [CLS] sentence1 [SEP] all have a token type ID of 0, while the other parts, corresponding to sentence2 [SEP], all have a token type ID of 1.

Note that if you select a different checkpoint, you won’t necessarily have the token_type_ids in your tokenized inputs (for instance, they’re not returned if you use a DistilBERT model). They are only returned when the model will know what to do with them, because it has seen them during its pretraining.

we will use the Dataset.map() method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset, so let’s define a function that tokenizes our inputs:

In [24]:
def tokenize_element(example): 
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

*Note* that we’ve left the padding argument out in our tokenization function for now. This is because padding all the samples to the maximum length is not efficient: it’s better to pad the samples when we’re building a batch, as then we only need to pad to the maximum length in that batch, and not the maximum length in the entire dataset. This can save a lot of time and processing power when the inputs have very variable lengths!

Here is how we apply the tokenization function on all our datasets at once. We’re using batched=True in our call to map so the function is applied to multiple elements of our dataset at once, and not on each element separately. This allows for faster preprocessing.

In [25]:
tokenized_datasets = raw_datasets.map(tokenize_element, batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

The way the 🤗 Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the preprocessing function. 

The last thing we will need to do is pad all the examples to the length of the longest element when we batch elements together — a technique we refer to as dynamic padding.

## Dynamic Padding 

A collate function combines individual samples into batches. While the default collator simply converts samples to tensors and concatenates them, this won't work when samples have different sizes. Instead of padding all samples to the same length upfront (which would be inefficient), padding can be applied only as needed within each batch.


The Transformers library provides a `DataCollatorWithPadding` class that handles this dynamic padding. When initialized with a tokenizer, it automatically adds the appropriate padding to batch items while respecting the model's padding requirements (token value and position). This approach improves training efficiency, though it may not be optimal for TPU training since TPUs perform better with fixed shapes.

In [26]:
from transformers import DataCollatorWithPadding

In [27]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

let’s grab a few samples from our training set that we would like to batch together. Here, we remove the columns idx, sentence1, and sentence2 as they won’t be needed and contain strings (and we can’t create tensors with strings) and have a look at the lengths of each entry in the batch:

In [28]:
samples = tokenized_datasets['train'][:8]
samples = {key:value for key, value in samples.items() if key not in ['idx', 'sentence1', 'sentence2']}
len(samples['input_ids'][0])
[len(x) for x in samples['input_ids']]

[50, 59, 47, 67, 59, 50, 62, 32]

No surprise, we get samples of varying length, from 32 to 67. Dynamic padding means the samples in this batch should all be padded to a length of 67, the maximum length inside the batch. Without dynamic padding, all of the samples would have to be padded to the maximum length in the whole dataset, or the maximum length the model can accept. Let’s double-check that our data_collator is dynamically padding the batch properly

In [29]:
batch = data_collator(samples)
{k:v.shape for k, v in batch.items()}

{'input_ids': TensorShape([8, 67]),
 'token_type_ids': TensorShape([8, 67]),
 'attention_mask': TensorShape([8, 67]),
 'labels': TensorShape([8])}

Now that we have our dataset and a data collator, we need to put them together. We could manually load batches and collate them, but that’s a lot of work, and probably not very performant either. Instead, there’s a simple method that offers a performant solution to this problem: to_tf_dataset(). This will wrap a tf.data.Dataset around your dataset, with an optional collation function. tf.data.Dataset is a native TensorFlow format that Keras can use for model.fit(), so this one method immediately converts a 🤗 Dataset to a format that’s ready for training.

In [30]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'], 
    label_cols=['labels'], 
    shuffle=True, 
    collate_fn=data_collator, 
    batch_size=8
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'], 
    label_cols=['labels'], 
    shuffle=False, 
    collate_fn=data_collator, 
    batch_size=8
)

/opt/anaconda3/envs/transformers/lib/python3.9/site-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


## Fine-Tuning a Model with Keras 

### Training 

In [31]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We get a warning after instantiating this pretrained model. This is because BERT has not been pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been inserted instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

To fine-tune the model on our dataset, we just have to compile() our model and then pass our data to the fit() method. This will start the fine-tuning process (which should take a couple of minutes on a GPU)

In [32]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [33]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer='adam', 
    loss=SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

In [34]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset)

459/459 [==============================] - 5340s 12s/step - loss: 0.6825 - accuracy: 0.6412 - val_loss: 0.6940 - val_accuracy: 0.3162


We can slowly reduce the learning rate over the course of training. In the literature, you will sometimes see this referred to as decaying or annealing the learning rate. In Keras, the best way to do this is to use a learning rate scheduler. A good one to use is PolynomialDecay — despite the name, with default settings it simply linearly decays the learning rate from the initial value to the final value over the course of training, which is exactly what we want.

In [35]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)


opt = Adam(learning_rate=lr_scheduler)

Now we have our all-new optimizer, and we can try training with it. First, let’s reload the model, to reset the changes to the weights from the training run we just did, and then we can compile it with the new optimizer

In [2]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

NameError: name 'TFAutoModelForSequenceClassification' is not defined

In [1]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

NameError: name 'model' is not defined